<a href="https://colab.research.google.com/github/B4ben-69/tensorFlow_edging/blob/main/Orgasm_mode_recording.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import asyncio
import websockets
import ssl
import json
from datetime import datetime
import argparse
import signal

# Handle asyncio event loop re-entry in Jupyter or similar environments
import nest_asyncio
nest_asyncio.apply()

# Global variable to handle cancellation
loop = asyncio.get_event_loop()
stop_event = asyncio.Event()

async def connect_to_websocket(uri, output_file):
    ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
    ssl_context.check_hostname = False
    ssl_context.verify_mode = ssl.CERT_NONE

    websocket = None
    file = None
    epoch = 0

    try:
        websocket = await websockets.connect(uri, ssl=ssl_context, ping_interval=None)

        # Send the request to start the stream
        start_stream_message = json.dumps({"streamReadings": 0})
        await websocket.send(start_stream_message)

        file = open(output_file, 'w')

        while not stop_event.is_set():
            try:
                message = await asyncio.wait_for(websocket.recv(), timeout=1)
                processed_message = process_message(message)
                epoch += 1
                if processed_message:
                    if epoch % 50 == 0:
                        print(f"{processed_message}")
                    file.write(processed_message + "\n")
            except asyncio.TimeoutError:
                continue
            except websockets.exceptions.ConnectionClosedError as e:
                print(f"Connexion fermée : {e}")
                break  # Exit the loop on disconnection
            except Exception as e:
                print(f"Erreur lors de la réception du message : {e}")
                break  # Exit the loop on other errors
    except asyncio.CancelledError:
        print("Task was cancelled, cleaning up...")
    except Exception as e:
        print(f"Erreur de connexion : {e}")
    finally:
        # Ensure the WebSocket connection is closed
        if websocket:
            await websocket.close()
            print("Connexion WebSocket fermée.")

        # Ensure the file is closed
        if file:
            file.close()
            print("Fichier de sortie fermé.")

def process_message(message):
    try:
        data = json.loads(message)
        if 'readings' in data:
            readings = data['readings']
            if readings['runMode'] != "ORGASM_MODE":
                return None
            processed_message = json.dumps(readings)
        else:
            processed_message = json.dumps(data)
    except json.JSONDecodeError:
        processed_message = message

    return processed_message

def shutdown(signal_received, frame):
    print("Signal reçu pour l'arrêt.")
    stop_event.set()
    for task in asyncio.all_tasks(loop):
        task.cancel()

if __name__ == "__main__":
    uri = "wss://" + "192.168.1.19"
    
    print(f"uri = {uri}")
    current_time = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = f"stream_output_{current_time}.txt"

    # Register the signal handler
    signal.signal(signal.SIGINT, shutdown)
    signal.signal(signal.SIGTERM, shutdown)

    try:
        loop.run_until_complete(connect_to_websocket(uri, output_file))
    except asyncio.CancelledError:
        pass
    finally:
        if not loop.is_running():
            loop.close()
        print("Programme arrêté.")
